In [ ]:
import glob
from matplotlib.pyplot import hist
import numpy as np
import skimage as ski
from matplotlib import pyplot as plt
import cv2
from skimage.feature import graycomatrix

# image = cv2.imread("chcek/cow/cow_1001.jpg", cv2.IMREAD_GRAYSCALE)

check_paths = glob.glob("check/**/*.jp*g", recursive=True)
test_paths = glob.glob("test/**/*.jp*g", recursive=True)

# Combine the lists
image_paths = check_paths + test_paths

for path in image_paths:
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    distance = 1
    angle = 7 * np.pi/4

    glcm = graycomatrix(
        image, distances=[distance], angles=[angle], levels=256, symmetric=True, normed=True
    )
    fig, ax = plt.subplots(2, 3, figsize=(12, 6))

    image_lbp = ski.feature.local_binary_pattern(image, P=256, R=1, method="uniform")

    fd, hog_image = ski.feature.hog(
        image,
        orientations=9,
        pixels_per_cell=(8,8),
        cells_per_block=(3,3),
        visualize=True,
        feature_vector=False,
    )

    hog_image_rescaled = ski.exposure.rescale_intensity(
        hog_image, in_range=(0, 1), out_range=(0, 1)
    )

    print(f"Shape of descriptors: {fd.shape}")
    print(f"Blocks: {fd.shape[0]}x{fd.shape[1]} = {fd.shape[0] * fd.shape[1]}")
    for i in range(fd.shape[0]):
        for j in range(fd.shape[1]):
            print(f"Block ({i}, {j}):")
    #        print(fd[i, j])

    plt.subplot(2, 3, 1)
    plt.imshow(image, cmap="gray")

    plt.subplot(2, 3, 2)
    plt.imshow(image_lbp, cmap="gray")

    plt.subplot(2, 3, 3)
    plt.imshow(glcm[:, :, 0, 0], cmap="gray")

    plt.subplot(2, 3, 4)
    plt.hist(
        image_lbp.ravel(),
        bins=np.arange(0, image_lbp.max() + 2) - 0.5,
        density=True
    )
    plt.title("LBP Histogram")
    plt.xlabel("LBP value")
    plt.ylabel("Frequency")

    plt.subplot(2, 3, 5)
    plt.imshow(hog_image_rescaled, cmap="gray")
    plt.title("HOG")

    plt.show()